In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
df=pd.read_csv('cleandanimeslyer.csv')
print(df.head())
print(df.describe())


   Unnamed: 0                                              title  eps_count  \
0           0                   Boku no Hero Academia 2nd Season       25.0   
1           1                                               Cue!       24.0   
2           2                       Maiko-san Chi no Makanai-san       12.0   
3           3  Dungeon ni Deai wo Motomeru no wa Machigatteir...       13.0   
4           5                            Golden Kamuy 4th Season       13.0   

  publish_date       status  season        studio  rate  eps_period    type  \
0      2023-10   compleated  Spring         Bones  8.10        23.0  series   
1      2023-10   compleated  Winter    Graphinica  6.79        24.0  series   
2      2023-12   compleated  Winter     J.C.Staff  7.03        25.0  series   
3      2023-10   compleated  Spring     J.C.Staff  7.54        24.0  series   
4      2023-09   compleated    Fall  Brain's Base  8.13        23.0  series   

                                      classificati

handle null and data types

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      1025 non-null   int64  
 1   title           1025 non-null   object 
 2   eps_count       959 non-null    float64
 3   publish_date    1001 non-null   object 
 4   status          1025 non-null   object 
 5   season          1025 non-null   object 
 6   studio          1025 non-null   object 
 7   rate            1025 non-null   float64
 8   eps_period      979 non-null    float64
 9   type            1025 non-null   object 
 10  classification  1025 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 88.2+ KB
None


In [5]:
df.drop(columns='Unnamed: 0',inplace=True)

In [6]:
na_val=df['eps_count'].isnull().sum()
na_val

66

In [7]:
# fill na for eps_count with 1 for movies and mean for series 
df.loc[(df['type'] == 'movie') & (df['eps_count'].isna()), 'eps_count'] = 1

In [8]:
mean_episodes = df[df['type'] == 'series']['eps_count'].mean(skipna=True)
print(int(mean_episodes))
df['eps_count'].fillna(int(mean_episodes),inplace=True)

17


In [9]:
df['eps_period'].fillna(int(df['eps_period'].mean()),inplace=True)

Analyse data

In [10]:
#top rated category
import ast 
df['classification'] = df['classification'].apply(ast.literal_eval)
df_exploded = df.explode('classification').rename(columns={'classification': 'classifications'})

In [11]:
#top ten classification animes 
genre_counts = df_exploded['classifications'].value_counts().reset_index()
genre_counts.columns = ['genre', 'count']
top_10_genres = genre_counts.head(10)
top_10_genres

,genre,count
0,Action,442
1,Fantasy,416
2,Comedy,305
3,Shounen,303
4,School,264
5,Adventure,235
6,Romance,217
7,Drama,193
8,Supernatural,130
9,Sci-Fi,127


In [12]:
fig = px.bar(
    top_10_genres,
    x='genre',
    y='count',
    text='count',
    title='Top 10 Most Popular Anime Categories',
    labels={'genre': 'Classification', 'count': 'Animes Count'},
    color='count',  
    color_continuous_scale='Viridis'
)


fig.update_layout(
    xaxis_tickangle=-45,  
    xaxis_title='Classification',
    yaxis_title='Animes Count',
    margin=dict(l=40, r=40, t=40, b=80),
    showlegend=False
)

fig.update_traces(texttemplate='%{text}', textposition='outside')


fig.show()

In [13]:
# rate per class 
# average of classification rate 
avg_rate_per_class=df_exploded.groupby('classifications')['rate'].mean().reset_index()
avg_rate_per_class=avg_rate_per_class.sort_values(by='rate',ascending=False)
top_avg_rate_classes=avg_rate_per_class.head(10)
top_avg_rate_classes

,classifications,rate
5,Award Winning,8.204118
8,Combat Sports,8.025714
69,Time Travel,7.993810
7,Childcare,7.968235
18,Gag Humor,7.951000
44,Parody,7.903913
42,Organized Crime,7.890476
29,Iyashikei,7.785909
59,Showbiz,7.725000
24,Historical,7.708788


In [17]:
fig=px.bar(
    top_avg_rate_classes,
    x='classifications',
    y='rate',
    text='rate',
    title='Top 10 rated classification ',
   
    color='rate',
    color_continuous_scale='Plasma',
    )
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title='classification',
    yaxis_title='avg_rate',
    margin=dict(l=40, r=40, t=40, b=80),
    showlegend=False
)
fig.show()

we can see that the classification that has less anime is highest rating ,
Keeping in mind that the large number is not an advantage here.
We also note that the highest-rated categories are not action or bloody.


In [22]:
# count of episodes and rate
eps_count_rate= df_exploded.groupby('eps_count')['rate'].mean().reset_index()
# eps_count_rate=eps_count_rate.sort_values('rate',ascending=False)
# eps_count_rate_top=eps_count_rate.head(15)
fig= px.line(
    eps_count_rate,
    x='eps_count',
    y='rate',
    title='Rating Vs Episodes count',
   
    
 
)
fig.show()

In [26]:

eps_count_rate= df_exploded.groupby('eps_period')['rate'].mean().reset_index()
fig= px.line(
    eps_count_rate,
    x='eps_period',
    y='rate',
    title='Rating Vs Episodes period',
    
   
    
 
)
fig.show()

In [ ]:
# anime per season
season_counts = df['season'].value_counts().reset_index()
season_counts.columns = ['season', 'count']
fig= px.pie(
    season_counts,
    names='season',
    values='count',
    color='season',
    
 )
fig.show()

In [38]:
#rate per season
rate_per_season=df.groupby('season')['rate'].mean().reset_index().sort_values('rate',ascending=False)
fig=px.bar(
    rate_per_season,
    x='season',
    y='rate',
    title='Rating Vs Season',
    color='season'
    
)
fig.show()